### Task 1 - Preproccess the data

In [ ]:
!pip install mne
!pip install -U mne-connectivity

In [ ]:
import numpy as np
import mne
from scipy.io import loadmat
from scipy.signal import welch
from scipy.stats import ttest_ind
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from mne.time_frequency import tfr_morlet, psd_array_multitaper
from scipy.signal import coherence
from mne_connectivity import spectral_connectivity_epochs

In [ ]:
# Load the EEG data from .mat files
healthy_data1 = loadmat('/content/H1.mat')
healthy_data2 = loadmat('/content/H2.mat')
healthy_data3 = loadmat('/content/H3.mat')

schizophrenia_data1 = loadmat('/content/Sch1.mat')
schizophrenia_data2 = loadmat('/content/Sch2.mat')
schizophrenia_data3 = loadmat('/content/Sch3.mat')

In [ ]:
# Create MNE Raw objects
sfreq = 250
CH_NUMBER = 19

healthy_raw1 = mne.io.RawArray(healthy_data1.get('H1'), mne.create_info(CH_NUMBER, sfreq, ch_types='eeg'))
healthy_raw2 = mne.io.RawArray(healthy_data2.get('H2'), mne.create_info(CH_NUMBER, sfreq, ch_types='eeg'))
healthy_raw3 = mne.io.RawArray(healthy_data3.get('H3'), mne.create_info(CH_NUMBER, sfreq, ch_types='eeg'))

schizophrenia_raw1 = mne.io.RawArray(schizophrenia_data1.get('Sch1'), mne.create_info(CH_NUMBER, sfreq, ch_types='eeg'))
schizophrenia_raw2 = mne.io.RawArray(schizophrenia_data2.get('Sch2'), mne.create_info(CH_NUMBER, sfreq, ch_types='eeg'))
schizophrenia_raw3 = mne.io.RawArray(schizophrenia_data3.get('Sch3'), mne.create_info(CH_NUMBER, sfreq, ch_types='eeg'))

# Concatenate the datasets
healthy_raw = mne.concatenate_raws([healthy_raw1, healthy_raw2, healthy_raw3])
schizophrenia_raw = mne.concatenate_raws([schizophrenia_raw1, schizophrenia_raw2, schizophrenia_raw3])

# Apply bandpass filter to remove high-frequency noise
healthy_raw.filter(l_freq=1, h_freq=50)  # Adjust the frequency range as needed
schizophrenia_raw.filter(l_freq=1, h_freq=50)

# Remove power line interference (50 Hz)
healthy_raw.notch_filter(freqs=50)
schizophrenia_raw.notch_filter(freqs=50)


Creating RawArray with float64 data, n_channels=19, n_times=231250
    Range : 0 ... 231249 =      0.000 ...   924.996 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=227500
    Range : 0 ... 227499 =      0.000 ...   909.996 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=227500
    Range : 0 ... 227499 =      0.000 ...   909.996 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=211250
    Range : 0 ... 211249 =      0.000 ...   844.996 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=286250
    Range : 0 ... 286249 =      0.000 ...  1144.996 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=241000
    Range : 0 ... 240999 =      0.000 ...   963.996 secs
Ready.
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-doma

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s


Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s


Filtering raw data in 3 contiguous segments
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth

...

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,19 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,1.00 Hz
Lowpass,50.00 Hz


### Task 2 - Segment the data into 2 second

In [ ]:
# Divide into 2-second segments
duration = 2  # in seconds
healthy_epochs = mne.make_fixed_length_epochs(healthy_raw, duration=duration)
schizophrenia_epochs = mne.make_fixed_length_epochs(schizophrenia_raw, duration=duration)

Not setting metadata
1372 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1477 matching events found
No baseline correction applied
0 projection items activated


### Task 3 - Extract
- Mean
- Standard derivation
- total signal power
- Alpha band power
- Beta band power
- theta band power
- fourier series coefficients

for each segment

In [ ]:

# Define frequency bands
freq_bands = {'theta': (4, 8),
              'alpha': (8, 13),
              'beta': (13, 30)}

# Function to calculate band power
def calculate_band_power(data, sfreq, freq_band):
    psd, freqs = mne.time_frequency.psd_array_welch(data, sfreq=sfreq, fmin=freq_band[0], fmax=freq_band[1])
    return np.sum(psd, axis=-1)

# Function to extract features from each segment
def extract_features(epochs):
    global sfreq
    features = []

    for epoch in epochs:
        data = epoch  # Get the EEG data for the epoch
        sfreq = sfreq

        # Calculate mean and standard deviation
        mean = np.mean(data, axis=-1, keepdims=True)
        std = np.std(data, axis=-1, keepdims=True)

        # Calculate total signal power
        total_power = np.sum(data ** 2, axis=-1, keepdims=True)

        # Calculate band powers
        alpha_power = calculate_band_power(data, sfreq, freq_bands['alpha'])
        beta_power = calculate_band_power(data, sfreq, freq_bands['beta'])
        theta_power = calculate_band_power(data, sfreq, freq_bands['theta'])

        # Flatten the band powers
        alpha_power = alpha_power.reshape(alpha_power.shape[0], -1)
        beta_power = beta_power.reshape(beta_power.shape[0], -1)
        theta_power = theta_power.reshape(theta_power.shape[0], -1)

        # Calculate Fourier series coefficients and flatten them
        fourier_coeffs = np.abs(np.fft.fft(data, axis=-1)).reshape(data.shape[0], -1)

        # Append features for the current segment
        segment_features = np.concatenate([mean, std, total_power, alpha_power, beta_power, theta_power, fourier_coeffs], axis=-1)
        features.append(segment_features)

    return np.array(features)

# Extract features for healthy and schizophrenia segments
healthy_features = extract_features(healthy_epochs)
schizophrenia_features = extract_features(schizophrenia_epochs)

Streaming output truncated to the last 5000 lines.
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (s)
Effective window size : 1.024 (

### Task 4 & 5
- Extract features for Each channel
- Implement feature matrix for both groups with Lable at the Last column

In [ ]:
healthy_data = healthy_epochs.get_data()
schizophrenia_data = schizophrenia_epochs.get_data()

WINDOW_SIZE = 100
OVERLAP = 0

# Create an array to store the labels (0 for healthy, 1 for schizophrenia)
num_channels = healthy_data.shape[1]
healthy_labels = np.zeros((len(healthy_epochs), 1, 1))  # Adjust the shape to (num_epochs, 1, 1)
schizophrenia_labels = np.ones((len(schizophrenia_epochs), 1, 1))  # Adjust the shape to (num_epochs, 1, 1)

# Ensure the shape of labels matches the number of epochs
healthy_labels_broadcasted = np.tile(healthy_labels, (1, num_channels, healthy_data.shape[2]))
schizophrenia_labels_broadcasted = np.tile(schizophrenia_labels, (1, num_channels, schizophrenia_data.shape[2]))

# Concatenate the data and labels along the last dimension
healthy_data_with_labels = np.concatenate([healthy_data, healthy_labels_broadcasted], axis=-1)
schizophrenia_data_with_labels = np.concatenate([schizophrenia_data, schizophrenia_labels_broadcasted], axis=-1)

# Concatenate the data from both groups along the first axis
all_data = np.concatenate([healthy_data_with_labels, schizophrenia_data_with_labels], axis=0)

# Create an array to store labels
labels = np.concatenate([np.zeros(len(healthy_epochs)), np.ones(len(schizophrenia_epochs))])

# Define frequency bands for PSD
freq_bands = {'alpha': [8, 13], 'beta': [13, 30], 'theta': [4, 8]}

# Create an array to store t-test results
t_test_results = np.zeros((num_channels, 5, len(freq_bands), all_data.shape[2] - 1))  # 5 features, 3 frequency bands, excluding the label column


# Iterate over channels
for channel in range(num_channels):
    # Iterate over features
    for feature in range(all_data.shape[2] - 1):  # Exclude the label column
        # Extract data for the current channel and feature
        healthy_channel_feature_data = all_data[:len(healthy_epochs), channel, feature]
        schizophrenia_channel_feature_data = all_data[len(healthy_epochs):, channel, feature]

        # 1. Time-domain Amplitudes
        _, t_test_results[channel, 0, :, feature] = ttest_ind(healthy_channel_feature_data, schizophrenia_channel_feature_data, axis=0, equal_var=False)

        # 2. Power Spectral Density (PSD)
        _, psd_healthy = welch(healthy_channel_feature_data, fs=sfreq)
        _, psd_schizophrenia = welch(schizophrenia_channel_feature_data, fs=sfreq)

        for i, (band, freq_range) in enumerate(freq_bands.items()):
            _, t_test_results[channel, 1, i, feature] = ttest_ind(np.sum(psd_healthy[freq_range[0]:freq_range[1]]), np.sum(psd_schizophrenia[freq_range[0]:freq_range[1]]), equal_var=False)

        # 3. Time-Frequency Analysis (Morlet Wavelet Transform)
        # Define parameters for Morlet Wavelet Transform
        freqs = np.arange(5, 50, 2)  # Frequency range of interest
        n_cycles = 4  # Adjust n_cycles to control wavelet length

        # Create MNE Epochs objects for Morlet Wavelet Transform
        info = mne.create_info([f'Channel_{ch}' for ch in range(num_channels)], sfreq, ch_types='eeg')
        epochs_healthy = mne.EpochsArray(healthy_data[:, :, :].reshape(len(healthy_epochs), num_channels, -1), info)
        epochs_schizophrenia = mne.EpochsArray(schizophrenia_data[:, :, :].reshape(len(schizophrenia_epochs), num_channels, -1), info)

        # Compute wavelet transform
        tfr_healthy = tfr_morlet(epochs_healthy, freqs=freqs, n_cycles=n_cycles, return_itc=False)
        tfr_schizophrenia = tfr_morlet(epochs_schizophrenia, freqs=freqs, n_cycles=n_cycles, return_itc=False)

        # Use get_data() to obtain the data array
        data_healthy = tfr_healthy.data
        data_schizophrenia = tfr_schizophrenia.data

        # Perform t-test on the resulting time-frequency data
        _, p_values = ttest_ind(data_healthy[:, channel, freq_bands['alpha'][0]:freq_bands['alpha'][1]], data_schizophrenia[:, channel, freq_bands['alpha'][0]:freq_bands['alpha'][1]], axis=0, equal_var=False)

        # Store the p-values in the t_test_results array
        t_test_results[channel, 2, 0, feature] = p_values[0]  # Assuming alpha band
        t_test_results[channel, 2, 1, feature] = p_values[1]  # Assuming beta band
        t_test_results[channel, 2, 2, feature] = p_values[2]  # Assuming theta band

        # 4. Connectivity Measures (Coherence)
        # Define coherence parameters
        fmin, fmax = 8, 13  # Frequency range of interest for coherence

        # Check that the data shapes match for coherence calculation
        # Check if the number of epochs match for coherence calculation
        assert healthy_channel_feature_data.shape[0] == schizophrenia_channel_feature_data.shape[0], "Data shapes do not match for coherence calculation."

        # Compute coherence
        _, _, S_healthy, S_schizophrenia, _ = spectral_connectivity_epochs([healthy_channel_feature_data, schizophrenia_channel_feature_data],
                                                                        method='coh', sfreq=sfreq, fmin=fmin, fmax=fmax, faverage=True)

        # Perform t-test on the resulting coherence data
        _, p_values = ttest_ind(S_healthy, S_schizophrenia, axis=0, equal_var=False)

        # Store the p-values in the t_test_results array
        t_test_results[channel, 3, :, feature] = p_values

        # 5. Event-Related Potentials (ERPs)
        _, t_test_results[channel, 4, :, feature] = ttest_ind(healthy_channel_feature_data, schizophrenia_channel_feature_data, axis=0, equal_var=False)


# Create a features matrix for each channel in each epoch
features_matrices = t_test_results.reshape((num_channels, -1))

# Print or use features_matrices for further analysis
print(features_matrices)
